# **TP 3 (Solution) : Spark SQL**

**Installation de pyspark :**

In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c0ebec4f9deab4f737ba98c7387c58af448429f602da276a0cba30a55ed98c2f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


1)	Créer un Dataframe à partir du fichier ngram.csv :

In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import Row

nomappli = "essai1"
config = SparkConf().setAppName(nomappli)
#sc = SparkContext(conf=config)
sc = SparkContext.getOrCreate();
sqlContext = SQLContext(sc)

lines = sc.textFile("/content/drive/MyDrive/TP03_spark/ngram.csv")
parts = lines.map(lambda l: l.split("\t"))
ngrams = parts.map(lambda x: Row(ngram=x[0], year=int(x[1]), ngramcount=int(x[2]), pages=int(x[3]), books=int(x[4])))
schemaNGrams = sqlContext.createDataFrame(ngrams)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


*2)	Enregistrer le Dataframe créé en tant que table temporaire :*

In [ ]:
schemaNGrams.registerTempTable("ngrams")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


3)	Répondre aux requêtes suivantes en utilisant deux méthodes : (1) le langage SQL, et (2) les méthodes de l’API SparkSQL.

3.1) Retourner tous les bi-grammes dont le nombre est supérieur à cinq :

---



In [ ]:
#SQL :
result = sqlContext.sql("SELECT ngram FROM ngrams WHERE ngramcount > 5")
result.show()

+--------+
|   ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



In [ ]:
#Les méthodes de l’API SparkSQL :
result = schemaNGrams.filter(schemaNGrams['ngramcount'] > 5).select('ngram')
result.show()

+--------+
|   ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



3.2) Retourner le nombre total de bi-grammes dans chaque année :

In [ ]:
#SQL :
result = sqlContext.sql("SELECT year, COUNT(*) AS cnt FROM ngrams GROUP BY year")
result.show()


+----+---+
|year|cnt|
+----+---+
|1806|  1|
|1840|  1|
|1882|  1|
|2007|  2|
|1887|  1|
|1991|  2|
|1858|  1|
|1833|  1|
|1831|  1|
|1987|  2|
|1999|  2|
|1997|  2|
|1884|  1|
|1845|  1|
|1877|  1|
|1829|  1|
|1867|  1|
|2006|  2|
|1876|  1|
|1879|  1|
+----+---+
only showing top 20 rows



In [ ]:
#Les méthodes de l’API SparkSQL :
result = schemaNGrams.groupBy("year").count()
result.show()

+----+-----+
|year|count|
+----+-----+
|1806|    1|
|1840|    1|
|1882|    1|
|2007|    2|
|1887|    1|
|1991|    2|
|1858|    1|
|1833|    1|
|1831|    1|
|1987|    2|
|1999|    2|
|1997|    2|
|1884|    1|
|1845|    1|
|1877|    1|
|1829|    1|
|1867|    1|
|2006|    2|
|1876|    1|
|1879|    1|
+----+-----+
only showing top 20 rows



3.3) Retourner les bi-grammes qui ont le plus grand nombre de count dans chaque année :

In [ ]:
#SQL :
result = sqlContext.sql("SELECT ngram, year, MAX(ngramcount) AS maxCount FROM ngrams GROUP BY ngram, year")
result.show()

+--------+----+--------+
|   ngram|year|maxCount|
+--------+----+--------+
|    ! 09|1880|       2|
|! $17.95|2008|       2|
|    ! 09|1845|       1|
|! $17.95|1995|       1|
|! $17.95|2004|      14|
|    ! 09|1780|       1|
|    ! 09|1833|       1|
|! $17.95|2007|       2|
|    ! 09|1872|       1|
|    ! 09|1861|       1|
|    ! 09|1806|       1|
|    ! 09|1882|       2|
|! $17.95|2001|       5|
|    ! 09|1881|       3|
|    ! 09|1884|       5|
|    ! 09|1823|       1|
|    ! 09|1848|       1|
|    ! 09|1887|       1|
|    ! 09|1803|       1|
|    ! 09|1877|       2|
+--------+----+--------+
only showing top 20 rows



In [ ]:
#Les méthodes de l’API SparkSQL :
result = schemaNGrams.groupBy("ngram", "Year").max("ngramcount")
result.show()

+--------+----+---------------+
|   ngram|Year|max(ngramcount)|
+--------+----+---------------+
|    ! 09|1880|              2|
|! $17.95|2008|              2|
|    ! 09|1845|              1|
|! $17.95|1995|              1|
|! $17.95|2004|             14|
|    ! 09|1780|              1|
|    ! 09|1833|              1|
|! $17.95|2007|              2|
|    ! 09|1872|              1|
|    ! 09|1861|              1|
|    ! 09|1806|              1|
|    ! 09|1882|              2|
|! $17.95|2001|              5|
|    ! 09|1881|              3|
|    ! 09|1884|              5|
|    ! 09|1823|              1|
|    ! 09|1848|              1|
|    ! 09|1887|              1|
|    ! 09|1803|              1|
|    ! 09|1877|              2|
+--------+----+---------------+
only showing top 20 rows



3.4) Retourner tous les bi-grammes qui sont apparus dans 80 années différentes :

In [ ]:
#SQL :
result = sqlContext.sql("SELECT ngram FROM ngrams GROUP BY ngram HAVING COUNT(DISTINCT Year) = 20").show()

+--------+
|   ngram|
+--------+
|! $17.95|
+--------+



In [ ]:
#Les méthodes de l’API SparkSQL :
result = schemaNGrams.groupBy("ngram").agg(
    countDistinct("Year").alias("NumYears")
).filter("NumYears = 20").select("ngram").show()

+--------+
|   ngram|
+--------+
|! $17.95|
+--------+



3.5) Retourner tous les bi-grammes qui contiennent le caractère ‘!’ dans la première partie et le caractère ‘9’ dans la deuxième partie (les deux parties sont séparées par un espace) :

In [ ]:
#SQL :
sqlContext.sql("SELECT distinct ngram FROM ngrams WHERE ngram LIKE '%!% %' AND ngram LIKE '% %9%'").show()

+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



In [ ]:
#Les méthodes de l’API SparkSQL :
schemaNGrams.filter((col("ngram").like("%!% %")) & (col("ngram").like("% %9%"))).select("ngram").distinct().show()

+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



3.6) Retourner les bi-grammes qui sont apparus dans toutes les années présentes dans les données :

In [ ]:
#SQL :
sqlContext.sql("SELECT ngram FROM ngrams GROUP BY ngram HAVING COUNT(DISTINCT Year) = (SELECT COUNT(DISTINCT Year) FROM ngrams)").show()

+-----+
|ngram|
+-----+
| ! 09|
+-----+



In [ ]:
#Les méthodes de l’API SparkSQL :
distinct_years = schemaNGrams.select("Year").distinct().count()

result = schemaNGrams.groupBy("ngram").agg(countDistinct("Year").alias("distinct_years")) \
    .filter("distinct_years = {}".format(distinct_years)) \
    .select("ngram").show()

+-----+
|ngram|
+-----+
| ! 09|
+-----+



3.7) Retourner le nombre total de pages et de livres dans lesquels chaque bi-gramme apparaît pour chaque année disponible, trié par ordre alphabétique :

In [ ]:
#SQL :
sqlContext.sql("SELECT Year, ngram, SUM(Pages) AS TotalPages, SUM(Books) AS TotalBooks FROM ngrams GROUP BY Year, ngram ORDER BY ngram ASC").show()

+----+--------+----------+----------+
|Year|   ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1998|! $17.95|         3|         3|
|1992|! $17.95|         5|         5|
|1995|! $17.95|         1|         1|
|1997|! $17.95|         5|         5|
|1993|! $17.95|         2|         2|
|1987|! $17.95|         1|         1|
|1985|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|2004|! $17.95|        14|        14|
|1996|! $17.95|         2|         2|
|2005|! $17.95|        13|        13|
|2001|! $17.95|         4|         4|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        10|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|2000|! $17.95|         9|         9|
|2002|! $17.95|         5|         5|
+----+--------+----------+----------+
only showing top 20 rows



In [ ]:
#Les méthodes de l’API SparkSQL :
result = (schemaNGrams
    .groupBy("Year", "ngram")
    .agg(sum("Pages").alias("TotalPages"), sum("Books").alias("TotalBooks"))
    .orderBy("ngram")
)
result.show()

+----+--------+----------+----------+
|Year|   ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1998|! $17.95|         3|         3|
|1992|! $17.95|         5|         5|
|1995|! $17.95|         1|         1|
|1997|! $17.95|         5|         5|
|1993|! $17.95|         2|         2|
|1987|! $17.95|         1|         1|
|1985|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|2004|! $17.95|        14|        14|
|1996|! $17.95|         2|         2|
|2005|! $17.95|        13|        13|
|2001|! $17.95|         4|         4|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        10|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|2000|! $17.95|         9|         9|
|2002|! $17.95|         5|         5|
+----+--------+----------+----------+
only showing top 20 rows



3.8) Retourner le nombre total de bi-grammes différents dans chaque année, triés par ordre décroissant de l'année :

In [ ]:
#SQL :
result = sqlContext.sql("SELECT Year, COUNT(DISTINCT ngram) AS distinct_bigrams FROM ngrams GROUP BY Year ORDER BY Year DESC")
result.show()

+----+----------------+
|Year|distinct_bigrams|
+----+----------------+
|2008|               2|
|2007|               2|
|2006|               2|
|2005|               2|
|2004|               2|
|2003|               2|
|2002|               2|
|2001|               2|
|2000|               2|
|1999|               2|
|1998|               2|
|1997|               2|
|1996|               2|
|1995|               2|
|1993|               2|
|1992|               2|
|1991|               2|
|1990|               2|
|1987|               2|
|1985|               2|
+----+----------------+
only showing top 20 rows



In [ ]:
#Les méthodes de l’API SparkSQL :
result = schemaNGrams.groupBy("Year").agg(countDistinct("ngram")) .orderBy("Year", ascending=False)
result.show()

+----+---------------------+
|Year|count(DISTINCT ngram)|
+----+---------------------+
|2008|                    2|
|2007|                    2|
|2006|                    2|
|2005|                    2|
|2004|                    2|
|2003|                    2|
|2002|                    2|
|2001|                    2|
|2000|                    2|
|1999|                    2|
|1998|                    2|
|1997|                    2|
|1996|                    2|
|1995|                    2|
|1993|                    2|
|1992|                    2|
|1991|                    2|
|1990|                    2|
|1987|                    2|
|1985|                    2|
+----+---------------------+
only showing top 20 rows

